In [ ]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
BASE_PATH = Path.cwd()
DATA_PATH = BASE_PATH /'data'
MAP_PATH = DATA_PATH / 'map'
CASE_PATH = DATA_PATH / 'case'

In [ ]:
case_before_2016 = pd.read_csv(CASE_PATH / 'case_before_2016.csv')

In [ ]:
case_after_2016 = pd.read_csv(CASE_PATH / 'case_after_2016.csv')

In [ ]:
case_before_2016

In [ ]:
cm = gpd.read_file('data/map/cnew.json')

In [ ]:
cm['Addr_Comm'] = cm['danhTuChun'] + ' ' + cm['diaDanh']

In [ ]:
cm['Addr_Comm'] = cm['Addr_Comm'].str.upper()

In [ ]:
comm_name = cm[['OBJECTID', 'Addr_Comm']]

In [ ]:
case_after_2016['Addr_Comm'] = case_after_2016['Addr_Comm'].apply(lambda x: 'XÃ ' + x if 'PHƯỜNG' not in x and 'THỊ TRẤN' not in x else x)
case_after_2016['Addr_Comm'] = case_after_2016['Addr_Comm'].str.replace('XÃ XÃ', 'XÃ')

In [ ]:
case_after_2016_m

In [ ]:
case_after_2016_m = pd.merge(comm_name, case_after_2016, on='Addr_Comm', how ='left')

In [ ]:
total_case = pd.merge(case_before_2016, case_after_2016_m, on=['OBJECTID']).fillna(0)

In [ ]:
other_columns = [col for col in total_case.columns if col not in ['OBJECTID', 'Addr_Comm']]

# Rearrange columns
new_columns_order = ['OBJECTID', 'Addr_Comm'] + other_columns
total_case = total_case[new_columns_order]

In [ ]:
#total_case.to_csv(CASE_PATH / 'case_final.csv', encoding='utf-8-sig', index=False)

In [ ]:
long_df = pd.melt(total_case, id_vars=['OBJECTID', 'Addr_Comm'], var_name='year', value_name='observed')

In [ ]:
long_df

In [ ]:
long_cm = pd.merge(cm, long_df, on=['OBJECTID','Addr_Comm'])

In [ ]:
long_cm.to_file(MAP_PATH / 'commune_w_case.json', driver='GeoJSON')

In [ ]:
gdf = long_cm.copy()

In [ ]:
year_data = gdf[gdf['year'] == '2015']

In [ ]:
gdf['year'] = gdf['year'].astype(int)

In [ ]:
gdf['observed'] = gdf['observed'].astype(int)

In [ ]:
# Load GeoJSON file into a GeoDataFrame


# Define the bins for categorizing observed values
bins = [0, 5, 10, float('inf')]
labels = ['0-5', '6-10', '>10']

# Create a map for each year
years = gdf['year'].unique()

# Set up subplot grid
num_rows = len(years) // 2 + (len(years) % 2)
fig, axs = plt.subplots(num_rows, 2, figsize=(15, 10))

# Plot each year's map
for i, year in enumerate(years):
    row = i // 2
    col = i % 2
    ax = axs[row, col] if num_rows > 1 else axs[col]
    
    # Filter data for the current year
    year_data = gdf[gdf['year'] == year]
    
    # Plot the data
    year_data.plot(column='observed')
    
    ax.set_title(f'Year {year}')
    ax.set_axis_off()

# Show the plot
plt.show()
